In [1]:
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
from stopwordsiso import stopwords
import re
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
import time
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.base import TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from scipy.stats import boxcox
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# Step 1: prepare data 

In [2]:
# df = pd.read_excel('data/Official_Provincial_Weibo_From_20191201_To_20200816.xlsx', engine='openpyxl')
# df.columns = ['id', 'bid', 'province', 'content', 'image_url', 'video_url', 'location', 'date', 'tool',
#        'like', 'comment', 'forward', 'topic', 'tag_user', 'padding']
# df.to_pickle('data/Official_Provincial_Weibo_From_20191201_To_20200816.pickle')
#df = pd.read_pickle('data/Official_Provincial_Weibo_From_20191201_To_20200816.pickle')
df1 = pd.read_pickle('data/Official_Provincial_Weibo_From_20191201_To_20200816.pickle')
df2 = pd.read_excel('data/0820.xlsx', engine='openpyxl')

In [3]:
df2.columns = df2.columns.str.lower()
df2['date'] = pd.to_datetime(df2.time_var,format='%Y%m%d')
df2 = df2.rename(columns = {'share':'forward'})
df2.province = df2.province.str.replace('\'','')

In [4]:
df1_sc = ['province', 'content', 'date', 'like', 'comment', 'forward']
df2_sc = ['province', 'like', 'forward', 'comment', 'freq', 'conf', 'susp', 'cure', 'dead', 'pop', 'gdp', 'followers', 'date']

In [5]:
df = df1.loc[:, df1_sc].merge(df2.loc[:, df2_sc], on = ['province', 'date', 'like', 'comment', 'forward'], how = 'inner')

In [6]:
scaler = MinMaxScaler()
def transformFeature(feature):
    featureP = boxcox((df[feature] + 1) / df.followers, 0)
    scaled_feture= np.squeeze(scaler.fit_transform(featureP.reshape(featureP.shape[0],1)))
    return scaled_feture

In [7]:
df['likeP'] = transformFeature('like') * 100
df['commentP'] = transformFeature('comment') * 100
df['forwardP'] = transformFeature('forward') * 100

In [8]:
df['content_re'] = df.content.apply(lambda x: re.sub(u"([^\u4e00-\u9fa5])","",x))
corpus = " ".join(jieba.cut(','.join(df.content_re)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/9m/b3fzzk753v9cswnwn8z8j7b80000gn/T/jieba.cache
Loading model cost 0.603 seconds.
Prefix dict has been built successfully.


## Static like/comment/forward prediction

In [9]:
X = np.concatenate((np.expand_dims(df.content_re.values, axis=1), 
                    pd.get_dummies(df.province).values), axis=1)
y = df.likeP

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
vectorizer = TfidfVectorizer(tokenizer=jieba.cut, 
                             stop_words=stopwords(["zh"]) | set([w for w in corpus if len(w) == 1]) | set(['借傥', '唷', '啷']), 
                             max_features = 100,
                             max_df = 0.9,
                             use_idf = False)
scaler = StandardScaler()

vectorized_content_train = vectorizer.fit_transform(X_train[:,0])
scaled_content_train = scaler.fit_transform(vectorized_content_train.todense())
vectorized_content_test = vectorizer.transform(X_test[:,0])
scaled_content_test = scaler.transform(vectorized_content_test.todense())

In [12]:
def evaluate_engagement(models, features):
    X = np.concatenate((np.expand_dims(df.content_re.values, axis=1), pd.get_dummies(df.province).values), axis=1)
    for model in models:
        for feature in features:
            y = df[feature + 'P']
            X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=42)
            gs = GridSearchCV(estimator=models[model]['model'],
                              param_grid=models[model]['params'],
                              scoring='neg_mean_squared_error',
                              n_jobs=6,
                              cv=5,
                              verbose=1)
            start = time.time()
            gs.fit(np.concatenate((scaled_content_train, X_train[:,1:]), axis=1), y_train) 
            end = time.time()
            print('Time to train model: %0.2fs' % (end -start))
            best_model = gs.best_estimator_
            y_pred = best_model.predict(np.concatenate((scaled_content_test, X_test[:,1:]), axis=1))
            print('label: {}, model: {}, MSE: {}'.format(feature, model, np.sqrt(mean_squared_error(y_test, y_pred))))

In [13]:
models = {'SGDRegressor': {'model': SGDRegressor(loss='squared_loss', penalty='l2', random_state=42, max_iter=100),
                           'params': {'penalty':['none','l2','l1'],'alpha':[1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 0.1]}},
          'RandomForestRegressor': {'model':RandomForestRegressor(random_state=42),
                                    'params':{'n_estimators':[100,200],'max_depth':[10,20,50,100]}},
          'MLPRegressor': {'model': MLPRegressor(random_state=42, max_iter=500, early_stopping = True, verbose = 1),
                           'params':{'hidden_layer_sizes':[(100,50,10), (100,100,100)],}}}

In [14]:
evaluate_engagement(models, ['like', 'comment', 'forward'])

Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    7.2s
[Parallel(n_jobs=6)]: Done 105 out of 105 | elapsed:   18.2s finished


Time to train model: 18.81s
label: like, model: SGDRegressor, MSE: 6.683538948446083
Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.4s
[Parallel(n_jobs=6)]: Done 105 out of 105 | elapsed:   17.4s finished


Time to train model: 18.11s
label: comment, model: SGDRegressor, MSE: 4.236275308718315
Fitting 5 folds for each of 21 candidates, totalling 105 fits


...

Iteration 1, loss = 101.88787014
Validation score: 0.211866
Iteration 2, loss = 33.51206472
Validation score: 0.371665
Iteration 3, loss = 27.19735908
Validation score: 0.489358
Iteration 4, loss = 22.90552637
Validation score: 0.558574
Iteration 5, loss = 20.73576584
Validation score: 0.591482
Iteration 6, loss = 19.72413399
Validation score: 0.604506
Iteration 7, loss = 19.04629324
Validation score: 0.619235
Iteration 8, loss = 18.61552242
Validation score: 0.621496
Iteration 9, loss = 18.36967719
Validation score: 0.625328
Iteration 10, loss = 18.11771526
Validation score: 0.625749
Iteration 11, loss = 17.80993492
Validation score: 0.631280
Iteration 12, loss = 17.55754976
Validation score: 0.631262
Iteration 13, loss = 17.35545506
Validation score: 0.629325
Iteration 14, loss = 17.24392041
Validation score: 0.635005
Iteration 15, loss = 17.08636530
Validation score: 0.635584
Iteration 16, loss = 16.94810364
Validation score: 0.633437
Iteration 17, loss = 16.81359900
Validation scor

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   21.0s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   21.0s finished


Iteration 1, loss = 81.72380796
Validation score: 0.342923
Iteration 2, loss = 26.19337209
Validation score: 0.575006
Iteration 3, loss = 15.06750241
Validation score: 0.792497
Iteration 4, loss = 8.66090334
Validation score: 0.850031
Iteration 5, loss = 7.08525659
Validation score: 0.868577
Iteration 6, loss = 6.56477176
Validation score: 0.871941
Iteration 7, loss = 6.32471573
Validation score: 0.876744
Iteration 8, loss = 6.17973317
Validation score: 0.879485
Iteration 9, loss = 6.04293719
Validation score: 0.876599
Iteration 10, loss = 5.94791715
Validation score: 0.880311
Iteration 11, loss = 5.86667555
Validation score: 0.878882
Iteration 12, loss = 5.76624877
Validation score: 0.881748
Iteration 13, loss = 5.69829794
Validation score: 0.879735
Iteration 14, loss = 5.64459967
Validation score: 0.881856
Iteration 15, loss = 5.59301851
Validation score: 0.882337
Iteration 16, loss = 5.56298635
Validation score: 0.873097
Iteration 17, loss = 5.54061867
Validation score: 0.881692
Ite

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   17.6s remaining:    0.0s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   17.6s finished


Iteration 1, loss = 82.09030214
Validation score: 0.353569
Iteration 2, loss = 26.23529007
Validation score: 0.586352
Iteration 3, loss = 15.06062532
Validation score: 0.800674
Iteration 4, loss = 8.68481482
Validation score: 0.856259
Iteration 5, loss = 7.12592721
Validation score: 0.873048
Iteration 6, loss = 6.59032698
Validation score: 0.875928
Iteration 7, loss = 6.33581163
Validation score: 0.878576
Iteration 8, loss = 6.21060135
Validation score: 0.883251
Iteration 9, loss = 6.05145268
Validation score: 0.885333
Iteration 10, loss = 5.97677317
Validation score: 0.883643
Iteration 11, loss = 5.90300982
Validation score: 0.881449
Iteration 12, loss = 5.77440603
Validation score: 0.885817
Iteration 13, loss = 5.73602958
Validation score: 0.883548
Iteration 14, loss = 5.65903177
Validation score: 0.887530
Iteration 15, loss = 5.60116755
Validation score: 0.887946
Iteration 16, loss = 5.54363485
Validation score: 0.873895
Iteration 17, loss = 5.57027042
Validation score: 0.881742
Ite

## Dynamic like/comment/forward prediction

In [15]:
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    mse = mean_squared_error(test, predictions)
    
    return mse

    # evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(features, p_values, d_values, q_values):
    for feature in features:
        dataset = df.groupby('date')[feature + 'P'].mean().values.astype('float32')
        best_score, best_cfg = float("inf"), None
        for p in p_values:
            for d in d_values:
                for q in q_values:
                    order = (p,d,q)
                    try:
                        mse = evaluate_arima_model(dataset, order)
                        if mse < best_score:
                            best_score, best_cfg = mse, order
                        print('ARIMA: %s MSE=%.3f' % (order,mse))
                    except:
                        continue
                    
        print('Label: %s Best ARIM: %s MSE=%.3f' % (feature, best_cfg, best_score))


In [16]:
p_values = [1, 3, 7,  14, 30]
d_values = range(0, 2)
q_values = range(0, 2)

evaluate_models(['like', 'comment', 'forward'], p_values, d_values, q_values)

ARIMA: (1, 0, 0) MSE=5.953
ARIMA: (1, 0, 1) MSE=3.891
ARIMA: (1, 1, 0) MSE=3.853
ARIMA: (1, 1, 1) MSE=3.601
ARIMA: (3, 0, 0) MSE=4.068
ARIMA: (3, 0, 1) MSE=3.915
ARIMA: (3, 1, 0) MSE=3.567
ARIMA: (3, 1, 1) MSE=3.584
ARIMA: (7, 0, 0) MSE=4.046
ARIMA: (7, 0, 1) MSE=4.051
ARIMA: (7, 1, 0) MSE=3.710
ARIMA: (7, 1, 1) MSE=3.732
ARIMA: (14, 0, 0) MSE=4.010
ARIMA: (14, 0, 1) MSE=4.011
ARIMA: (14, 1, 0) MSE=3.667
ARIMA: (14, 1, 1) MSE=3.698
ARIMA: (30, 0, 0) MSE=4.089


/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimiz

/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimiz

ARIMA: (30, 0, 1) MSE=4.281
ARIMA: (30, 1, 0) MSE=3.796
ARIMA: (30, 1, 1) MSE=3.806
Label: like Best ARIM: (3, 1, 0) MSE=3.567
ARIMA: (1, 0, 0) MSE=6.353
ARIMA: (1, 0, 1) MSE=2.755
ARIMA: (1, 1, 0) MSE=2.543
ARIMA: (1, 1, 1) MSE=2.667
ARIMA: (3, 0, 0) MSE=3.550
ARIMA: (3, 0, 1) MSE=2.772
ARIMA: (3, 1, 0) MSE=2.455
ARIMA: (3, 1, 1) MSE=2.487
ARIMA: (7, 0, 0) MSE=2.796


/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimiz

...

/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimiz

ARIMA: (30, 0, 1) MSE=3.223
ARIMA: (30, 1, 0) MSE=3.148


/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimiz

ARIMA: (30, 1, 1) MSE=3.152
Label: forward Best ARIM: (3, 1, 0) MSE=2.455


/usr/local/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [17]:
mlp_df = pd.concat([df.groupby('date').content_re.sum(), 
                     df.groupby('date').likeP.mean(),
                     df.groupby('date').commentP.mean(),
                     df.groupby('date').forwardP.mean()], axis = 1).reset_index()

In [18]:
size = int(len(mlp_df) * 0.8)

In [19]:
vectorizer = TfidfVectorizer(tokenizer=jieba.cut, 
                             stop_words=stopwords(["zh"]) | set([w for w in corpus if len(w) == 1]) | set(['借傥', '唷', '啷']), 
                             max_features = 50,
                             max_df = 0.9,
                             use_idf = False)
scaler = StandardScaler()

vectorized_content_train = vectorizer.fit_transform(mlp_df.content_re[:size])
scaled_content_train = scaler.fit_transform(vectorized_content_train.todense())
vectorized_content = vectorizer.transform(mlp_df.content_re)
scaled_content = scaler.transform(vectorized_content.todense())

In [20]:
def split_sequence(sequence, n_step):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_step
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = np.concatenate((sequence[i:end_ix,1:].flatten(), sequence[i:end_ix - 1,0])), sequence[end_ix, [0]]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [25]:
def train_model(x_train, y_train, x_test, y_test, n_feature):
    model = Sequential()
    model.add(Dense(n_feature, activation='relu', input_shape=(n_feature,)))
    model.add(Dense(n_feature, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(x_train, y_train, epochs=100, verbose=0)
    predictions = model.predict(x_test, verbose=1)
    mse = mean_squared_error(y_test, predictions)
    return mse

In [26]:
def evaluate_models(features, n_steps):
    for feature in features:
        X = np.concatenate((np.expand_dims(mlp_df[feature + 'P'].values, 1), scaled_content), axis = 1)
        train, test = X[0:size], X[size:len(X)]
        best_score, best_cfg = float("inf"), None
        for n_step in n_steps:
            n_feature = n_step * 50 + n_step - 1
            X_train, y_train = split_sequence(train, n_step)
            X_test, y_test = split_sequence(test, n_step)
            mse = train_model(X_train, y_train, X_test, y_test, n_feature)
            order = n_step
            if mse < best_score:
                best_score, best_cfg = mse, order
            #print('MLP: %s MSE=%.3f' % (n_step, mse))
        print()
        print('Label: %s Best MLP: %s MSE=%.3f' % (feature, best_cfg, best_score))
        print()

In [28]:
evaluate_models(['like', 'comment', 'forward'], [1, 3, 7,  14, 30])

1/1 [==============================] - 0s 464us/step

Label: like Best MLP: 30 MSE=2.856

1/1 [==============================] - 0s 560us/step

Label: comment Best MLP: 30 MSE=2.645



1/1 [==============================] - 0s 600us/step

Label: forward Best MLP: 30 MSE=2.894

